In [1]:
import pandas as pd
from datetime import datetime

#### Set target date of report export

In [10]:
#Set target date of report exports
reportDate= datetime(2022, 1, 14)
print('Report Date:', reportDate.date(), '\n')

Report Date: 2022-01-14 



#### Calculate metrics from Backlog report

In [11]:
#Import necessary reports
fusion= pd.read_csv('today_report.csv')
fcrClose= pd.read_csv('Tier 1 Closed- Last 7 Days-2022-01-14.csv')
fcrEsc= pd.read_csv('Tier 1 Escalations- Last 7 days-2022-01-14.csv')

#Calculate total backlog as a sum of Functional Backlog and Development backlog
devBacklog= len(fusion[fusion['Department'] == 'CF Development'])
funcBacklog= len(fusion['Case Number'] - devBacklog)
totalBacklog= funcBacklog + devBacklog

#Calculate aging metrics as an average of 'Age' and 'Last Modified'
avgAge = fusion['Age (Days)'].mean()
avgAgeUpdate = (reportDate - pd.to_datetime(fusion['Case Date/Time Last Modified'])).mean()

#### Calculate Tier 1 FCR from 'Last 7 Days' reports

In [4]:
#Subset table by desired agents
tier1Closed= sum(
    fcrClose['Edited By'].isin(['Lona Balcar', 'Amanda Massingill','Juan Pineda',
                                'Dennis Key','Sara Galindo','Derek Powell']))

#Subset table by desired agents
tier1Esc= sum(
    fcrEsc['Edited By'].isin(['Lona Balcar', 'Amanda Massingill','Juan Pineda','Dennis Key',
                              'Sara Galindo','Derek Powell'])
    & ~fcrEsc['New Value'].isin(['Tier 1 - County Fusion','Tier 1.5 - County Fusion']))

#Take user input on number of Chats from external source
chats = int(input("Number of Chats:"))

#Calculate FCR% as a ratio of tier1Esc to the overall
tier1FCR = tier1Esc / (tier1Closed + tier1Esc + chats)

Number of Chats:734


## Print Fusion Metrics

In [13]:
#Print metrics
print('Total Backlog:', totalBacklog)
print('Functional Fusion Backlog:', funcBacklog)
print('Development Backlog:', devBacklog)
print('Average Age:', int(avgAge))
print('Average Age Since Last Update:', avgAgeUpdate.days, '\n')

#Print FCR metrics
print('Tier 1 Closed- Past 7 Days:', tier1Closed)
print('Tier 1 Escalated- Past 7 Days:', tier1Esc)

print('\nTier 1 FCR% (w/ Chats):', round(1 - tier1Esc / (tier1Esc + tier1Closed + chats),2))
print('Tier 1 FCR% (w/o Chats):', round(1- tier1Esc / (tier1Esc + tier1Closed),2))

Total Backlog: 771
Functional Fusion Backlog: 698
Development Backlog: 73
Average Age: 176
Average Age Since Last Update: 49 

Tier 1 Closed- Past 7 Days: 103
Tier 1 Escalated- Past 7 Days: 81

Tier 1 FCR% (w/ Chats): 0.91
Tier 1 FCR% (w/o Chats): 0.56


#### Calculate Vanguard Metrics from VG Backlog Report

In [6]:
#Import needed backlog report
vgReport= pd.read_csv('VG Backlog-2022-01-14.csv')

#Count total cases as a sum of cases where TP ID ~null and cases where TP ID isany
devBacklog= sum(vgReport['TP ID'].value_counts())
funcBacklog= len(vgReport['Case Number']) - devBacklog
totalBacklog= funcBacklog + devBacklog

#subset cases without TP ID and take averages for age
avgAge = vgReport[vgReport['TP ID'].isna()]['Age'].mean()
avgAgeUpdate = vgReport[vgReport['TP ID'].isna()]['Number of Days Since Last Activity'].mean()

#subset cases created in the last week with a TP ID for later calculation of FCR using Chats#
esctoTP= len(vgReport[(pd.to_datetime(vgReport['Date/Time Opened']) > '2022-01-06') &
                 ~vgReport['TP ID'].isna()])

#### Calculate Chat metrics from Slack raw data

In [7]:
#Import Chats table from local source, 'rawSheet'
rawSheet = pd.read_csv('Support Data From Slack - 2022 Data.csv')

#Subset raw data to only chats received during the defined timeline, 'lastWeekChats'
lastWeekChats= rawSheet[(pd.to_datetime(rawSheet['Date Posted']) > '2022-01-06') & 
                      (pd.to_datetime(rawSheet['Date Posted']) < '2022-01-14')]

#Calculate average 'Time to Respond' (ASA) for Chats, 'chatsASA'
chatsASA= pd.to_datetime(lastWeekChats['Time to Respond'], format='%H:%M:%S').mean()

#Calculate first-chat resolution (FCR) as a ratio of chats:esctoTP 
vgFCR= esctoTP / len(lastWeekChats['Channel'])

## Print Vanguard Metrics

In [8]:
print('Total Backlog:', totalBacklog)
print('Functional Backlog:', funcBacklog)
print('Development Backlog:', devBacklog)
print('Average Age:', round(avgAge))
print('Average Age Since Last Update:', round(avgAgeUpdate))
print('VG FCR:', round(1- vgFCR, 2),'%')
print('Chats ASA:', chatsASA.minute, 'minutes', chatsASA.second, 'seconds')

Total Backlog: 114
Functional Backlog: 4
Development Backlog: 110
Average Age: 18
Average Age Since Last Update: 5
VG FCR: 0.95 %
Chats ASA: 2 minutes 26 seconds
